# [0] DEV

In [1]:
# Load DGL Graph
import dgl

graph, _ = dgl.load_graphs('test_graph.bin')
graph = graph[0]

In [2]:
graph

Graph(num_nodes=87345, num_edges=0,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'pos': Scheme(shape=(3,), dtype=torch.float32), 'sv': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(2,), dtype=torch.float32), 'node_type': Scheme(shape=(1,), dtype=torch.float32)}
      edata_schemes={'edge_type': Scheme(shape=(1,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int32), 'x': Scheme(shape=(8,), dtype=torch.float32)})

### Follow the Tutorial UPT

In [3]:
# check successful setup
import torch
print(torch.__version__)
device = torch.device("cuda")
torch.cuda.get_device_name(device)
from torch_geometric.nn.pool import radius_graph
from torch_scatter import segment_csr

2.3.0+cu121


In [4]:
# general imports
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from tqdm import tqdm

#### HyperParams

In [5]:
dim = 192 # What is this DIM?
num_heads = 3
epochs = 1000
batch_size = 1

In [7]:
from upt.models.upt import UPT
from upt.models.approximator import Approximator
from upt.models.decoder_perceiver import DecoderPerceiver
from upt.models.encoder_supernodes import EncoderSupernodes

model = UPT(
    encoder=EncoderSupernodes(
        # simulation has 7 inputs: node_type, normals, sv
        input_dim=7,
        # 3D dataset
        ndim=3,
        # positions are rescaled to [0, 200]
        radius=5,
        # in regions where there are a lot of mesh cells, it would result in supernodes having a lot of
        # connections to nodes. but since we sample the supernodes uniform, we also have a lot of supernodes
        # in dense regions, so we can simply limit the maximum amount of connections to each supernodes
        # to avoid an extreme amount of edges
        max_degree=32,
        # dimension for the supernode pooling -> use same as ViT-T latent dim
        gnn_dim=dim,
        # ViT-T latent dimension
        enc_dim=dim,
        enc_num_heads=num_heads,
        # ViT-T has 12 blocks -> parameters are split evenly among encoder/approximator/decoder
        enc_depth=4,
        # downsample to 128 latent tokens for fast training
        perc_dim=dim,
        perc_num_heads=num_heads,
        num_latent_tokens=128,
    ),
    approximator=Approximator(
        # tell the approximator the dimension of the input (perc_dim or enc_dim of encoder)
        input_dim=dim,
        # as in ViT-T
        dim=dim,
        num_heads=num_heads,
        # ViT-T has 12 blocks -> parameters are split evenly among encoder/approximator/decoder
        depth=4,
    ),
    decoder=DecoderPerceiver(
        # tell the decoder the dimension of the input (dim of approximator)
        input_dim=dim,
        # CpT + Cp
        output_dim=2,
        # simulation is 3D
        ndim=3,
        # as in ViT-T
        dim=dim,
        num_heads=num_heads,
        # ViT-T has 12 blocks -> parameters are split evenly among encoder/approximator/decoder
        depth=4,
        # we assume num_outputs to be constant so we can simply reshape the dense result into a sparse tensor
        unbatch_mode="dense_to_sparse_unpadded",
    ),
)
model = model.to(device)
print(f"parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

TypeError: UPT.__init__() missing 1 required positional argument: 'conditioner'